# Rodando C++ com CPLEX através do makefile criado

In [ ]:
ls

In [ ]:
! make clean

In [ ]:
ls

In [ ]:
ls instancias/

In [ ]:
! make ti ARGS=instancias/instancia_10projetos_2sondas_delta_t28.dat

# Metaheurísticas

## Lendo dados

In [1]:
def Read_data(path):
    
    import copy
    
    coords_x = []
    coords_y = []
    bacias = []
    nomes = []
    maturidades = []
    qualidades = []
    plays = []
    soterramentos = []
    pcgnas = []
    geracao = []
    migracao = []
    reservatorio = []
    geometria = []
    retencao = []
    pshc = []
    mc_vol = []
    mi_vol = []
    mc_vpl = []
    mi_vpl = []
    custos = []
    tempos_exec = []
    inicio_janela = []
    final_janela = []
    sondas_x = []
    sondas_y = []
    
    data = {
        1: coords_x,
        2: coords_y,
        3: bacias,
        4: nomes,
        5: maturidades,
        6: qualidades,
        7: plays,
        8: soterramentos,
        9: pcgnas,
        10: geracao,
        11: migracao,
        12: reservatorio,
        13: geometria,
        14: retencao,
        15: pshc,
        16: mc_vol,
        17: mi_vol,
        18: mc_vpl,
        19: mi_vpl,
        20: custos,
        21: tempos_exec,
        22: inicio_janela,
        23: final_janela,
        25: sondas_x,
        26: sondas_y
    }
    
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if i == 0:
                n_projetos = int(line.split()[0])
                custo_total = float(line.split()[1])
                t_init = int(line.split()[2])
                t_final = int(line.split()[3])
                delta_t = int(line.split()[4])
                n_periodos = int(line.split()[5])
            elif i == 24:
                n_sondas = int(line.split()[0])
            elif i <= 26:
                for elem in line.split('[')[1].split(']')[0].split(','):
                    try:
                        data[i].append(float(elem))
                    except:
                        data[i].append(elem)
            elif i > 26:
                setup = []
                for k, l in enumerate(line.split('[[')[1].split('], [')):
                    setup.append([])
                    for elem in l.split(']]')[0].split(', '):
                        setup[k].append(float(elem))
    
    return n_projetos, n_sondas, n_periodos, t_init, t_final, delta_t, custo_total, data, setup


## Classe para lista duplamente encadeada

In [2]:
class Node:
    
    """
    Documentação:
    """
    
    def __init__(self, data=None):
        self.data = data
        self.next = None
        self.prev = None
    

class DoublyLinkedList:
    
    """
    Documentação:
    """
    
    def __init__(self, nodes=None):
        self.head = None
        if nodes is not None:
            node = Node(data=nodes.pop(0))
            self.head = node
            for elem in nodes:
                new_node = Node(data=elem)
                node.next = new_node
                new_node.prev = node
                node = node.next
    
    def __iter__(self):
        node = self.head
        while node is not None:
            yield node
            node = node.next
    
    def add_node_begining(self, value_to_add):
        new_node = Node(data=value_to_add)
        
        if not self.head:
            self.head = new_node
            return
        
        new_node.next = self.head
        self.head.prev = new_node
        self.head = new_node
    
    def add_node_end(self, value_to_add):
        
        new_node = Node(data=value_to_add)
        
        if not self.head:
            self.head = new_node
            return
        
        for curr in self:
            pass
        curr.next = new_node
        new_node.prev = curr
    
    def add_node_after_node(self, value_to_add, node_ref):
        new_node = Node(data=value_to_add)
        new_node.next = node_ref.next
        new_node.prev = node_ref
        if (node_ref.next != None):
            node_ref.next.prev = new_node
        node_ref.next = new_node
    
    def add_node_before_node(self, value_to_add, node_ref):
        new_node = Node(data=value_to_add)
        if node_ref == self.head:
            self.head.prev = new_node
            new_node.next = self.head
            self.head = new_node
        else:
            node_ref.prev.next = new_node
            new_node.prev = node_ref.prev
            new_node.next = node_ref
            node_ref.prev = new_node
    
    def remove_node_by_ref(self, node):
        if node == self.head:
            node.next.prev = None
            self.head = node.next
            del node
        else:
            node.next.prev = node.prev
            node.prev.next = node.next
            del node
    
    def find_value(self, value):
        for node in self:
            if node.data == value: break
        return node

# Classe para Heap

In [3]:
class MinHeap(object):
    
    def __init__(self, modo=None, array=None):
        if modo == None:
            self.dados = []
        if modo == 'array':
            self.Heapfy(array)
    
    def Ajustar_para_baixo(self, i):
        left_child = 2 * i + 1
        right_child = 2 * i + 2
        length = len(self.dados) - 1
        smallest = i
        if left_child <= length and self.dados[i] > self.dados[left_child]:
            smallest = left_child
        if right_child <= length and self.dados[smallest] > self.dados[right_child]:
            smallest = right_child
        if smallest != i:
            self.dados[i], self.dados[smallest] = self.dados[smallest], self.dados[i]
            self.Ajustar_para_baixo(smallest)
    
    def Ajustar_para_cima(self, i):
        parent = math.floor(i / 2)
        length = len(self.dados) - 1
        if parent >= 0 and self.dados[parent] > self.dados[i]:
            self.dados[i], self.dados[parent] = self.dados[parent], self.dados[i]
            self.Ajustar_para_cima(parent)
    
    def Heapfy(self, array):
        self.dados = array.copy()
        for i in reversed(range(len(array)//2)):
            self.Ajustar_para_baixo(i)
    
    def GetMin(self):
        return self.dados[0]
    
    def PopMin(self):
        self.dados[0], self.dados[-1] = self.dados[-1], self.dados[0]
        minimo = self.dados.pop()
        self.Ajustar_para_baixo(0)
        return minimo 
    
    def InsertItem(self, value):
        self.dados.append(value)
        self.Ajustar_para_cima(len(self.dados) - 1)
    
    def PopItem(self, index):
        self.dados[index], self.dados[-1] = self.dados[-1], self.dados[index]
        self.dados.pop()
        if index < len(self.dados):
            self.Ajustar_para_cima(index)
            self.Ajustar_para_baixo(0)
        return
    
    def HeapSort(self, array):
        self.dados = array.copy()
        self.Heapfy(self.dados)
        sorted_array = []
        for _ in range(len(self.dados)):
            self.dados[0], self.dados[-1] = self.dados[-1], self.dados[0]
            sorted_array.append(self.dados.pop())
            self.Ajustar_para_baixo(0)
        return sorted_array

# Classe para árvore de busca binária

In [4]:
class NodeArvoreBinaria(object):
    """
    documentação da classe
    """
    def __init__(self, label, key, h_left=None, h_right=None):
        
        self.label = label
        self.key = key
        self.index = None
        self.parent = None
        self.left_child = None
        self.right_child = None
        if h_left != None: self.h_left = h_left
        if h_right != None: self.h_right = h_right
    
    def CopiarNode(self):
        key = self.GetKey()
        label = self.GetLabel()
        right_child = self.GetRightChild()
        left_child = self.GetLeftChild()
        parent = self.GetParent()
        
        novo_node = NodeArvoreBinaria(label, key)
        
        novo_node.right_child = right_child
        novo_node.left_child = left_child
        novo_node.parent = parent
        
        return novo_node
    
    def GetKey(self):
        if self == None: return None
        return self.key
    
    def SetKey(self, key):
        self.key = key
    
    def GetLabel(self):
        if self == None: return None
        return self.label
    
    def SetLabel(self, label):
        self.label = label
    
    def GetLeftChild(self):
        return self.left_child
    
    def SetLeftChild(self, child):
        self.left_child = child
    
    def GetRightChild(self):
        return self.right_child
    
    def SetRightChild(self, child):
        self.right_child = child
    
    def GetParent(self):
        return self.parent
    
    def SetParent(self, parent):
        self.parent = parent
    
    def Copiar_Dados(self, target):
        self.SetKey(target.GetKey())
        self.SetLabel(target.GetLabel())
    
    def Copiar_Filhos(self, target):
        self.SetLeftChild(target.GetLeftChild())
        self.SetRightChild(target.GetRightChild())
    
    def Deletar_Right_Child(self):
        filho = self.GetRightChild()
        self.right_child.SetParent(None)
        self.SetRightChild(None)
        del filho
    
    def Deletar_Left_Child(self):
        filho = self.GetLeftChild()
        self.left_child.SetParent(None)
        self.SetLeftChild(None)
        del filho
    

class Arvore_Binaria_de_Busca(object):
    """
    documentação da classe
    """
    def __init__(self):
        self.tamanho = 0
        self.root = None
    
    def GetRoot(self):
        return self.root
    
    def SetRoot(self, root):
        self.root = root
    
    def DeletarLeaf(self, node):
        if node.GetParent() == None:
            self.SetRoot(None)
            self.tamanho = 0
            del node
            return
        else:
            self.tamanho -= 1
            if node.GetKey() <= node.parent.GetKey():
                node.parent.Deletar_Left_Child()
            elif node.GetKey() > node.parent.GetKey():
                node.parent.Deletar_Right_Child()
            del node
    
    def DeletarPaiDeFilhoUnico(self, node):
        if node.GetKey() == self.root.GetKey():
            if node.GetLeftChild() != None:
                node.left_child.SetParent(None)
                self.SetRoot(node.left_child)
            elif node.GetRightChild() != None:
                node.right_child.SetParent(None)
                self.SetRoot(node.right_child)
        else:
            if node.GetKey() <= node.parent.GetKey():
                if node.GetLeftChild() != None:
                    node.left_child.SetParent(node.parent)
                    node.parent.SetLeftChild(node.left_child)
                elif node.GetRightChild() != None:
                    node.right_child.SetParent(node.parent)
                    node.parent.SetLeftChild(node.right_child)
            if node.GetKey() > node.parent.GetKey():
                if node.GetLeftChild() != None:
                    node.left_child.SetParent(node.parent)
                    node.parent.SetRightChild(node.left_child)
                elif node.GetRightChild() != None:
                    node.right_child.SetParent(node.parent)
                    node.parent.SetRightChild(node.right_child)
        del node
        self.tamanho -= 1
    
    def DeletarPaiDeDois(self, node, sucessor):
        if sucessor == node.right_child:
            sucessor.SetParent(node.parent)
            sucessor.SetLeftChild(node.left_child)
            node.left_child.SetParent(sucessor)
            
            if node.GetKey() == self.root.GetKey():
                self.SetRoot(sucessor)
            else:
                if node.GetKey() <= node.parent.GetKey():
                    node.parent.SetLeftChild(sucessor)
                elif node.GetKey() > node.parent.GetKey():
                    node.parent.SetRightChild(sucessor)
        else:
            if sucessor.GetRightChild() != None:
                sucessor.right_child.SetParent(sucessor.parent)
                sucessor.parent.SetLeftChild(sucessor.right_child)
            else:
                sucessor.parent.SetLeftChild(None)
            
            sucessor.SetParent(node.parent)
            sucessor.SetLeftChild(node.left_child)
            node.left_child.SetParent(sucessor)
            sucessor.SetRightChild(node.right_child)
            node.right_child.SetParent(sucessor)
            
            if node.GetKey() == self.root.GetKey():
                self.SetRoot(sucessor)
            else:
                if node.GetKey() <= node.parent.GetKey():
                    node.parent.SetLeftChild(sucessor)
                elif node.GetKey() > node.parent.GetKey():
                    node.parent.SetRightChild(sucessor)
        self.tamanho -= 1
        del node
    
    def Busca(self, valor, start='root'):
        
        # se nao for especificado o início da busca, começar pela raíz
        if start == 'root': start = self.GetRoot()
        
        # se não tiver raíz, retorna None
        if start == None:
            return 'arvore vazia'
        
        # se valor encotrado, retornar
        if start.GetKey() == valor:
            res_busca = {'found': True, 'node':start, 'key': start.GetKey(), 'label': start.GetLabel(), 'indice': start.index, 'parent': start.GetParent()}
            return res_busca
        
        # se valor não encontrado
        else:
            if valor < start.GetKey():
                start = start.GetLeftChild()
                if start == None:
                    res_busca = {'found': False, 'parent': start}
                    found = True
                    return res_busca
                else:
                    return self.Busca(valor, start=start)
            else:
                if start.GetRightChild() == None:
                    res_busca = {'found': False, 'parent': start}
                    found = True
                    return res_busca
                else:
                    start = start.GetRightChild()
                    return self.Busca(valor, start=start)
    
    def Inserir(self, label, key, start='root'):
        
        if start == 'root': start = self.root
        
        novo_node = NodeArvoreBinaria(label, key)
        
        # se a arvore está vazia
        if self.root == None:
            self.SetRoot(novo_node)
            novo_node.index = 0
            self.tamanho = 1
        # se a árvore já tem, pelo menos, a raíz
        else:
            if key <= start.GetKey():
                if start.GetLeftChild() == None:
                    start.SetLeftChild(novo_node)
                    novo_node.SetParent(start)
                    novo_node.index = start.index + 1
                    self.tamanho += 1
                else:
                    self.Inserir(label, key, start=start.left_child)
            elif key > start.GetKey():
                if start.GetRightChild() == None:
                    start.SetRightChild(novo_node)
                    novo_node.SetParent(start)
                    novo_node.index = start.index + 1
                    self.tamanho += 1
                else:
                    self.Inserir(label, key, start=start.right_child)
    
    def GetMin(self):
        if self.GetRoot() == None: return None
        start = self.root
        minimo = self.root.GetKey()
        min_node = start
        while start.GetLeftChild() != None:
            start = start.left_child
            if start.GetKey() < minimo:
                minimo = start.GetKey()
                min_node = start
        return min_node, minimo
    
    def GetMax(self):
        if self.GetRoot() == None: return None
        start = self.root
        maximo = self.root.GetKey()
        max_node = start
        while start.GetRightChild() != None:
            start = start.right_child
            if start.GetKey() > maximo:
                maximo = start.GetKey()
                max_node = start
        return max_node, maximo
    
    def GetSucessor(self, start):
        if start.GetRightChild() == None: return None
        sucessor = start.right_child
        while sucessor.GetLeftChild() != None:
            sucessor = sucessor.left_child
        return (sucessor, sucessor.GetKey())
    
    def Deletar(self, valor, start='root'):
        
        if start == 'root': start = self.root
        
        # se não achar valor
        if start == None: return start
        
        # se tem que ir pra esquerda
        if valor < start.GetKey():
            self.Deletar(valor, start=start.left_child)
        # se tem que ir pra direita
        elif valor > start.key:
            self.Deletar(valor, start.right_child)
        # se encontrou o valor, deletar
        else:
            # caso 1: se node já está numa folha, simplesmente remover
            if start.GetLeftChild() == None and start.GetRightChild() == None:
                self.DeletarLeaf(start)
                return
            
            # caso 2: se só tem um filho, filho substitui
            elif not(start.GetLeftChild() != None and start.GetRightChild() != None):
                self.DeletarPaiDeFilhoUnico(start)
                return
            
            # caso 3: se tem dois filhos, achar sucessor e substituir pelo node a ser deletado
            else:
                sucessor = self.GetSucessor(start)[0]
                self.DeletarPaiDeDois(start, sucessor)
                return
    
    def PopMin(self):
        min_node, minimo = self.GetMin()
        label = min_node.GetLabel()
        self.Deletar(minimo)
        return {'label': label, 'key': minimo}
    
    def PopMax(self):
        max_node, maximo = self.GetMax()
        label = max_node.GetLabel()
        self.Deletar(maximo)
        return {'label': label, 'key': maximo}

# Outras estruturas de dados

In [5]:
import Fibonacci_Heap

In [6]:
import Arvore_AVL

## Implementação da heurística de construção

In [7]:
def ConstruirSolucao(dados, n_projetos, n_sondas, n_periodos, custo_total, desloc, criterio='lucro/custo'):
    
    import math
    import copy
    import numpy as np
    import heapq
    
    lag = n_sondas
    
    # inicializando solução
    s = {i:DoublyLinkedList(nodes=[[-1, 0, n_periodos]]) for i in range(n_sondas)}
    
    # inicializando lista de candidatos, por sonda
    s_candidatos = {i:[] for i in range(n_sondas)}
    
    # armazena lista de candidatos como uma heap, dependendo do critério guloso
    for i in range(n_sondas):
        for j in range(n_projetos):
            if (criterio == 'lucro'):
                criterio_val = dados[19][j]
                heapq.heappush( s_candidatos[i], (-criterio_val, j) )
            elif (criterio == 'lucro/custo'):
                criterio_val = dados[19][j] / dados[20][j]
                heapq.heappush( s_candidatos[i], (-criterio_val, j) )
            elif (criterio == 'lucro/duracao'):
                criterio_val = dados[19][j] / (dados[21][j]+1)
                heapq.heappush( s_candidatos[i], (-criterio_val, j) )
            elif (criterio == 'custo'):
                criterio_val = dados[20][j]
                heapq.heappush( s_candidatos[i], (criterio_val, j) )
            elif (criterio == 'duracao'):
                criterio_val = dados[21][j]
                heapq.heappush( s_candidatos[i], (criterio_val, j) )
    
    # inicializa contador de gastos total
    gastos = 0
    
    # inicializa conjunto de projetos já selecionados
    proj_usados = set()
    
    # conjunto de sondas é representado com uma Heap
    sondas = [(-n_periodos, i) for i in range(n_sondas)]
    
    # inicializa valor da função objetivo da solução
    fitness = 0
    
    # enquanto nenhuma restrição for alcançada
    while (gastos < custo_total and sondas):
        
        # escolher uma sonda
        (tam_prev, sonda) = heapq.heappop(sondas)
        
        # escolher um projeto
        projeto = heapq.heappop(s_candidatos[sonda])[1]
        
        # se valor do projeto não aumenta a função objetivo, pular
        if (dados[19][projeto] <= 0.):
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, (tam_prev, sonda))
            continue
        
        # se projeto já foi escolhido, pular
        if (projeto in proj_usados):
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, (tam_prev, sonda))
            continue
        
        # se executar o projeto excede limite de gastos, pular
        if (gastos + dados[20][projeto] > custo_total):
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, (tam_prev, sonda))
            continue
        
        aloc = False
        proc = dados[21][projeto]
        
        # percorre atividades da sonda para ver se existe janela disponível na sonda
        for i, node in enumerate(s[sonda]):
            
            # se node não for de janela de tempo, pular
            if (node.data[0] != -1):
                continue
            
            # busca tempo de deslocamento / setup 
            # se node for head, considerar origem da sonda
            if node.prev == None:
                prev = sonda
            # senão, considerar projeto anterior
            else:
                prev = node.prev.data[0] + lag
            setup = desloc[prev][projeto + lag]
            
            # se existe projeto alocado após a janela atual, o setup para ele vai mudar quanto?
            if node.next != None:
                old_setup = desloc[prev][node.next.data[0] + lag]
                new_setup = desloc[projeto + lag][node.next.data[0] + lag]
                delta_setup = old_setup - new_setup
            else:
                delta_setup = 0
            
            # calcula o mínimo tempo de início e o máximo tempo de término do projeto dentro da janela
            release = max(node.data[1], dados[22][projeto])
            due = min(node.data[2] + delta_setup, dados[23][projeto])
            
            # se consigo colocar o projeto dentro da janela
            if (release + proc + setup -1 <= due):
                
                # escolher tempo de inicio para alocar projeto
                delta1 = max((release - node.data[1]), (due - release) - (proc + setup) + (node.data[2] - due))
                delta2 = max((node.data[2] - due), (due - release) - (proc + setup) + (release - node.data[1]))
                if (delta1 >= delta2):
                    inicio = copy.copy(release)
                    final = copy.copy(release + setup + proc - 1)
                else:
                    inicio = copy.copy(due - proc - setup + 1)
                    final = copy.copy(due)
                
                # atualiza release do next e due da janela, caso o setup tenha sido alterado
                if delta_setup != 0:
                    # se for reduzir o deslocamento, não tem problema de viabilidade nem conflito
                    if (delta_setup > 0):
                        if (node.next.data[2] > node.next.data[1]):
                            node.next.data[1] += delta_setup
                            node.data[2] += delta_setup
                            aloc = True
                    # se for aumentar, tentar primeiro colocar diferença pra trás
                    elif (delta_setup < 0):
                        # posso reduzir a release do projeto que sofre modificação do deslocamento? 
                        # testar sua janela de tempo (viabilidade)
                        flag_tw = 'inviavel'
                        if (node.next.data[1] + delta_setup >= dados[22][node.next.data[0]]):
                            flag_tw = 'viavel'
                        flag_conflict = 'inviavel'
                        # testar conflito com projeto a ser inserido
                        if (node.next.data[1] + delta_setup > final):
                            flag_conflict = 'viavel'
                        
                        # ações para cada combinação de flag e marcar flag aloc
                        # se não tem problema de janela nem de conflito, segue o jogo
                        if ((flag_tw=='viavel') and (flag_conflict=='viavel')):
                            node.next.data[1] += delta_setup
                            node.data[2] += delta_setup
                            aloc = True
                        # senão, se não tem problema de janela, mas tem conflito
                        elif ((flag_tw=='viavel') and (flag_conflict=='inviavel')):
                            # teria que alterar inicio e final do projeto a ser inserido
                            # se continua dentro da janela 
                            if (inicio+delta_setup >= node.data[1]):
                                # e é viável pela janela de tempo
                                if (inicio+delta_setup >= dados[22][node.data[0]]):
                                    inicio += delta_setup
                                    final += delta_setup
                                    node.next.data[1] += delta_setup
                                    node.data[2] += delta_setup
                                    aloc = True
                            # senão, se é viável pela janela de tempo
                            elif (inicio+delta_setup >= dados[22][node.data[0]]):
                                # e consigo antecipar o prev
                                if (node.prev != None):
                                    if (node.prev.prev != None):
                                        if (node.prev.prev.data[0] == -1):
                                            # quanto consigo antecipar o prev?
                                            delta_prev = node.prev.data[1] - max(node.prev.prev.data[1], dados[22][node.prev.data[0]])
                                            delta_prev = max(0, delta_prev)
                                            # se é suficiente
                                            if (delta_prev >= -delta_setup):
                                                node.prev.prev.data[2] += delta_setup
                                                node.prev.data[1] += delta_setup
                                                node.prev.data[2] += delta_setup
                                                inicio += delta_setup
                                                final += delta_setup
                                                node.next.data[1] += delta_setup
                                                node.data[2] += delta_setup
                                                aloc = True
                        
                        # se ainda não foi alocado, tentar colocar diferença para frente
                        if (not aloc):
                            # posso aumentar a due do projeto que sofre modificação do deslocamento?
                            # testar sua janela de tempo (viabilidade)
                            flag_tw = 'inviavel'
                            if (node.next.data[2] - delta_setup <= dados[23][node.next.data[0]]):
                                flag_tw = 'viavel'
                            # testar conflito com projeto next
                            flag_conflict = 'inviavel'
                            if (node.next.next != None):
                                if (node.next.next.data[0] == -1):
                                    flag_conflict = 'viavel'

                            # ações para cada combinação de flag e marcar flag aloc
                            # se não tem problema de janela nem conflito, segue o jogo
                            if ((flag_tw == 'viavel') and (flag_conflict == 'viavel')):
                                node.next.data[2] -= delta_setup
                                node.next.next.data[1] -= delta_setup
                                aloc = True
                            # senão, se não tem problema de janela, mas tem de conflito
                            elif ((flag_tw == 'viavel') and (flag_conflict == 'inviavel')):
                                # teria que tentar postergar projeto next next
                                if (node.next.next.next != None):
                                    if (node.next.next.next.data[0] == -1):
                                        if (node.next.next.data[2] - delta_setup <= dados[23][node.next.next.data[0]]):
                                            node.next.next.next.data[1] -= delta_setup
                                            node.next.next.data[2] -= delta_setup
                                            node.next.next.data[1] -= delta_setup
                                            node.next.data[2] -= delta_setup
                                            aloc = True
                    if (not aloc):
                        break
                
                print ('projeto ', projeto, ' sendo inserido na sonda', sonda, ' de ', inicio, ' até ', final,
                       ', sendo setup=', setup, ', e proc=', proc, ' delta_setup=', delta_setup)
                
                # alocar
                proj_usados.add(copy.copy(projeto))
                fitness += dados[19][projeto]
                aloc = True
                
                # atualizando a lista de atividades ========================================
                # se projeto preenche janela toda
                if ( (int(node.data[1])==int(inicio)) and (int(node.data[2])==int(final)) ):
                    
                    # substituir janela pela alocação do projeto
                    node.data = [copy.copy(projeto), inicio, final]
                
                # senão, se projeto preenche início da janela    
                elif (int(node.data[1])==int(inicio)):
                    
                    # atualizar release da janela
                    node.data[1] = final + 1
                    
                    # inserir projeto antes da janela
                    s[sonda].add_node_before_node(value_to_add=[projeto, inicio, final], node_ref=node)
                
                # senão, se projeto preenche final da janela
                elif (int(node.data[2])==int(final)):
                    
                    # atualiza a due da janela
                    node.data[2] = inicio - 1
                    
                    # inserir projeto depois da janela
                    s[sonda].add_node_after_node(value_to_add=[projeto, inicio, final], node_ref=node)
                
                # senão, se projeto preenche o meio da janela
                else:
                    
                    # atualiza release da janela
                    temp = copy.copy(node.data[1])
                    node.data[1] = final + 1
                    
                    # insere janela nova antes da janela original
                    s[sonda].add_node_before_node(value_to_add=[-1, temp, inicio - 1], node_ref=node)
                    
                    # insere projeto antes da janela original
                    s[sonda].add_node_before_node(value_to_add=[projeto, inicio, final], node_ref=node)
                    
                break
            else:
                # senão, tentar realocar projetos ====================================================
                
                # verificar se janela da sonda tem interceção com janela do projeto
                if ((node.data[1] <= dados[23][projeto]) and (node.data[2] >= dados[22][projeto])):
                    
                    # consigo antecipar o prev? quanto?
                    delta_prev = 0
                    if node.prev == None:
                        delta_prev = 0
                    elif node.prev.prev == None:
                        delta_prev = 0
                    else:
                        if node.prev.prev.data[0] != -1:
                            delta_prev = 0
                        else:
                            delta_prev = node.prev.data[1] - max(node.prev.prev.data[1], dados[22][node.prev.data[0]])
                            delta_prev = max(0, delta_prev)
                    
                    # consigo postergar o next? quanto?
                    delta_next = 0
                    if node.next == None:
                        delta_next = 0
                    elif node.next.next == None:
                        delta_next = 0
                    else:
                        if node.next.next.data[0] != -1:
                            delta_next = 0
                        else:
                            delta_next = min(node.next.next.data[2], dados[23][node.next.data[0]]) - node.next.data[2]
                            delta_next = max(0, delta_next)
                    
                    # quanto de delta eu preciso?
                    delta_min = (setup + proc) - (node.data[2] - node.data[1] + 1) - delta_setup
                    
                    # só com antecipação do prev, torna viável?
                    if (delta_prev >= delta_min):
                        # se contempla janela
                        if ((node.data[1]-delta_min>=dados[22][projeto]) and (node.data[2]+delta_setup<=dados[23][projeto])):
                            flag_realoc = 'viavel'
                            # se há mudança de setup para mais
                            if (delta_setup < 0):
                                # checar viabilidade de diminuir release do next
                                if (node.next.data[1] + delta_setup < dados[22][node.next.data[0]]):
                                    flag_realoc = 'inviavel'
                            if (flag_realoc == 'viavel'):
                                # fazer antecipação do prev
                                node.prev.data[1] -= delta_min
                                node.prev.data[2] -= delta_min
                                node.prev.prev.data[2] -= delta_min
                                aloc = True
                                node.data[1] -= delta_min
                        
                    # só com postergação do next, torna viável?
                    elif (delta_next >= delta_min):
                        # se contempla janela
                        if ((node.data[1]>=dados[22][projeto]) and (node.data[2]+delta_min+delta_setup<=dados[23][projeto])):
                            flag_realoc = 'viavel'
                            # se há mudança de setup para mais
                            if (delta_setup < 0):
                                # checar viabilidade de diminuir release do next
                                if (node.next.data[1] + delta_setup < dados[22][node.next.data[0]]):
                                    flag_realoc = 'inviavel'
                            if (flag_realoc == 'viavel'):
                                # fazer postergação do next
                                node.next.data[1] += delta_min
                                node.next.data[2] += delta_min
                                node.next.next.data[1] += delta_min
                                aloc = True
                                node.data[2] += delta_min
                    
                    # somando antecipação do prev e postergação do next, torna viável?
                    elif (delta_prev + delta_next >= delta_min):
                        # se, antecipando prev primeiro, contempla janela
                        if ((node.data[1]-delta_prev>=dados[22][projeto]) and (node.data[2]+delta_setup+(delta_min - delta_prev)<=dados[23][projeto])):
                            flag_realoc = 'viavel'
                            # se há mudança de setup
                            if (delta_setup < 0):
                                # checar viabilidade de diminuir release do next
                                if (node.next.data[1] + delta_setup < dados[22][node.next.data[0]]):
                                    flag_realoc = 'inviavel'
                            if (flag_realoc == 'viavel'):
                                # fazer antecipação do prev primeiro
                                node.prev.data[1] -= delta_prev
                                node.prev.data[2] -= delta_prev
                                node.prev.prev.data[2] -= delta_prev
                                node.data[1] -= delta_prev
                                # fazer postergação do next depois
                                node.next.data[1] += (delta_min - delta_prev)
                                node.next.data[2] += (delta_min - delta_prev)
                                node.next.next.data[1] += (delta_min - delta_prev)
                                aloc = True
                                node.data[2] += (delta_min - delta_prev)
                        # senão, se, postergando nexto primeiro, contempla janela
                        elif ((node.data[1]-(delta_min - delta_next)>=dados[22][projeto]) and (node.data[2]+delta_setup+delta_next<=dados[23][projeto])):
                            flag_realoc = 'viavel'
                            # se há mudança de setup
                            if (delta_setup < 0):
                                # checar viabilidade de diminuir release do next
                                if (node.next.data[1] + delta_setup < dados[22][node.next.data[0]]):
                                    flag_realoc = 'inviavel'
                            if (flag_realoc == 'viavel'):
                                # fazer postergação do next primeiro
                                node.next.data[1] += delta_next
                                node.next.data[2] += delta_next
                                node.next.next.data[1] += delta_next
                                node.data[2] += delta_next
                                # fazer antecipação do prev depois
                                node.prev.data[1] -= (delta_min - delta_next)
                                node.prev.data[2] -= (delta_min - delta_next)
                                node.prev.prev.data[2] -= (delta_min - delta_next)
                                aloc = True
                                node.data[1] -= (delta_min - delta_next)
                            
                    # alocar projeto
                    if aloc:
                        
                        print ('realocando: projeto ', projeto, ' sendo inserido na sonda', sonda, 
                               ' na janela de ', node.data[1], ' até ', node.data[2], 'com delta_min=', delta_min,
                               ' delta_prev=', delta_prev, ' delta_next=', delta_next, ' e delta_setup', delta_setup,
                               ', sendo setup=', setup, ', e proc=', proc)
                        
                        # atualiza release do next e due da janela, caso o setup tenha sido alterado
                        if delta_setup != 0:
                            node.next.data[1] += delta_setup
                            node.data[2] += delta_setup
                        
                        node.data[0] = projeto
                        proj_usados.add(copy.copy(projeto))
                        fitness += dados[19][projeto]
                        break
        
        # se projeto foi alocado, atualizar dados
        if (aloc):
            gastos += dados[20][projeto]
            # se sonda ainda tem candidatos, colocar de volta na heap com valor atualizado
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, ( (tam_prev + (final-inicio+1)) , sonda))
        # senão, apenas recolocar a sonda
        else:
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, (tam_prev, sonda))
    
    return fitness, gastos, s, s_candidatos

In [8]:
def VerificaFuncaoObjetivo(s, valor, dados):
    
    """
    Documentação:
    """
    
    fitness = 0
    for sonda in s.keys():
        for node in s[sonda]:
            #print ('verificando sonda ', sonda, ' e node ', node)
            if node.data[0] != -1:
                fitness += dados[19][node.data[0]]
    
    if round(fitness, 5) == round(valor, 5):
        print ('função objetivo correta')
    else:
        print ('função objetivo incorreta: ', valor, ' reportado, e ', fitness, ' apurado')
    
    return round(fitness, 5) == round(valor, 5)

In [9]:
def VerificaRestricaoOrcamento(s, valor, dados):
    
    """
    Documentação:
    """
    
    custo = 0
    for sonda in s.keys():
        for node in s[sonda]:
            #print ('verificando sonda ', sonda, ' e node ', node)
            if node.data[0] != -1:
                custo += dados[20][node.data[0]]
    
    if round(custo, 5) == round(valor, 5):
        print ('orçamento correto')
    else:
        print ('orçamento incorreto: ', valor, ' reportado, e ', custo, ' apurado')
    
    return round(custo, 5) == round(valor, 5)

In [10]:
def VerificaRestricaoUnicidadeProjetos(s, dados):
    
    """
    Documentação:
    """
    
    result = True
    projetos = set()
    for sonda in s.keys():
        for node in s[sonda]:
            #print ('verificando sonda ', sonda, ' e node ', node)
            if node.data[0] != -1:
                if node.data[0] in projetos:
                    result = False
                    print ('unicidade incorreta: projeto ', node.data[0], ' duplicado')
                    return result
                else:
                    projetos.add(node.data[0])
    
    print ('unicidade correta')
    
    return result

In [11]:
def VerificaRestricaoTimeWindows(s, dados):
    
    """
    Documentação:
    """
    
    result = True
    for sonda in s.keys():
        for node in s[sonda]:
            #print ('verificando sonda ', sonda, ' e node ', node)
            if node.data[0] != -1:
                if not ((node.data[1] >= dados[22][node.data[0]])  and (node.data[1] <= dados[23][node.data[0]])):
                    result = False
                    print ('janelas de tempo incorretas: projeto ', node.data[0], ' tem contrato de ', 
                          dados[22][node.data[0]], ' a ', dados[23][node.data[0]], ' e está alocado de ', 
                          node.data[1], ' a ', node.data[2])
                    return result
    
    print ('janelas de tempo corretas')
    
    return result

In [12]:
def VerificaRestricaoScheduling(s, dados, desloc):
    
    """
    Documentação:
    """
    
    lag = len(s.keys())
    
    result = True
    for sonda in s.keys():
        for node in s[sonda]:
            #print ('verificando sonda ', sonda, ' e node ', node.data[0], ' node_prev', node.prev)
            if node.data[0] != -1:
                
                # se projeto vem após origem da sonda
                if (node.prev == None):
                    setup = desloc[sonda][node.data[0]+lag]
                elif ( (node.prev.data[0] == -1) and (node.prev.prev == None) ):
                    setup = desloc[sonda][node.data[0]+lag]
                # senão, projeto vem após algum outro projeto
                else:
                    # se existe janela de sonda antes do projeto
                    if node.prev.data[0] == -1:
                        setup = desloc[node.prev.prev.data[0]+lag][node.data[0]+lag]
                    # senão, ele vem imediatamente após o outro projeto
                    else:
                        setup = desloc[node.prev.data[0]+lag][node.data[0]+lag]
                
                if (node.data[2]-node.data[1] + 1 != setup + dados[21][node.data[0]]):
                    result = False
                    print ('setup incorreto: projeto ', node.data[0], ' alocado de ', 
                           node.data[1], ' a ', node.data[2], ', sendo setup=', setup, ' e proc. time=', 
                          dados[21][node.data[0]], ' logo, ', node.data[2]-node.data[1]+1, ' != ', 
                          setup + dados[21][node.data[0]])
                    return result
    
    print ('setup correto')
    
    return result

In [13]:
def VerificaRestricoes(s, dados, valor_fit, valor_custos, desloc):
    
    """
    Documentação:
    """
    
    fo = VerificaFuncaoObjetivo(s, valor_fit, dados)
    
    orc = VerificaRestricaoOrcamento(s, valor_custos, dados)
    
    unic = VerificaRestricaoUnicidadeProjetos(s, dados)
    
    time_wind = VerificaRestricaoTimeWindows(s, dados)
    
    sched = VerificaRestricaoScheduling(s, dados, desloc)
    
    return fo & orc & unic & time_wind & sched

In [14]:
def RodarHeuristica(arquivo, criterio='lucro/custo', print_sol=True):
    
    """
    Documentação:
    """
    
    import CPUtimer
    import copy
    
    n_projetos, n_sondas, n_periodos, t_init, t_final, delta_t, custo_total, dados, desloc = Read_data(path=arquivo)
    
    timer = CPUtimer.CPUTimer()
    timer.reset()
    timer.start()
    
    f, gastos , s, _ = ConstruirSolucao(dados, n_projetos, n_sondas, n_periodos, custo_total, desloc, criterio=criterio)
    
    check = VerificaRestricoes(s, dados, f, gastos, desloc)
    
    tempo_total = timer.get_time(reference = "total", unit = "seconds")
    tempo_medio = timer.get_time(reference = "average", unit = "seconds")
    
    if check:
        check_str = 'feasible'
    else:
        check_str = 'not feasible'
    
    if print_sol:
        print (check_str)
        print ('fitness: ', f)
        print ('gastos: ', gastos)
        print ('tempo total de execução (segundos) : ', tempo_total)
        print ('tempo médio de execução (segundos) ', tempo_medio)
        for i in range(n_sondas):
            for node in s[i]:
                print ('sonda ', i, ' executa projetos: ', node.data)
    
    return f, gastos, s, tempo_total, tempo_medio, check_str

In [15]:
def RodarHeuristicaSensibilidade(diretorio, criterios=None):
    
    """
    Documentação:
    """
    
    from os import walk
    import copy
    import pandas as pd
    
    if criterios == None:
        criterios = ['lucro', 'lucro/custo', 'lucro/duracao', 'custo', 'duracao']
    
    resultados = {}
    resultados['Instancia'] = []
    resultados['Viabilidade'] = []
    resultados['Criterio guloso'] = []
    resultados['Tempo total (s)'] = []
    resultados['Tempo medio (s)'] = []
    resultados['Valor da função objetivo'] = []
    
    arquivos = _, _, arquivos = next(walk(diretorio))
    arquivos = ["./instancias/" + f for f in arquivos]
    
    for f in arquivos:
        
        print ('processando arquivo: ', f)
        
        for c in criterios:
            fitness, gastos, s, t_total, t_medio, feasible = RodarHeuristica(arquivo=f, criterio=c, print_sol=False)
            
            resultados['Instancia'].append(copy.deepcopy(f.split(diretorio)[1]))
            resultados['Viabilidade'].append(copy.deepcopy(feasible))
            resultados['Criterio guloso'].append(copy.deepcopy(c))
            resultados['Tempo total (s)'].append(copy.deepcopy(t_total))
            resultados['Tempo medio (s)'].append(copy.deepcopy(t_medio))
            resultados['Valor da função objetivo'].append(copy.deepcopy(fitness))
    
    df = pd.DataFrame(resultados)
    
    return resultados, df

In [16]:
ls instancias/

instancia_1000projetos_10sondas_delta_t1.dat
instancia_1000projetos_10sondas_delta_t28.dat
instancia_1000projetos_10sondas_delta_t7.dat
instancia_1000projetos_20sondas_delta_t1.dat
instancia_1000projetos_20sondas_delta_t28.dat
instancia_1000projetos_20sondas_delta_t7.dat
instancia_1000projetos_5sondas_delta_t1.dat
instancia_1000projetos_5sondas_delta_t28.dat
instancia_1000projetos_5sondas_delta_t7.dat
instancia_100projetos_10sondas_delta_t1.dat
instancia_100projetos_10sondas_delta_t28.dat
instancia_100projetos_10sondas_delta_t7.dat
instancia_100projetos_2sondas_delta_t1.dat
instancia_100projetos_2sondas_delta_t28.dat
instancia_100projetos_2sondas_delta_t7.dat
instancia_100projetos_5sondas_delta_t1.dat
instancia_100projetos_5sondas_delta_t28.dat
instancia_100projetos_5sondas_delta_t7.dat
instancia_10projetos_2sondas_delta_t1.dat
instancia_10projetos_2sondas_delta_t28.dat
instancia_10projetos_2sondas_delta_t7.dat
instancia_200projetos_10sondas_delta_t1.dat
instancia_200projetos_10sondas_

In [17]:
arq = 'instancias/instancia_100projetos_10sondas_delta_t1.dat'

In [18]:
n_projetos, n_sondas, n_periodos, t_init, t_final, delta_t, custo_total, dados, desloc = Read_data(path=arq)

In [19]:
print ('setup=', desloc[80+n_sondas][97+n_sondas])

setup= 77.0


In [20]:
print ('setup=', desloc[97+n_sondas][37+n_sondas])

setup= 68.0


In [21]:
print ('proc=', dados[21][97])

proc= 200.0


In [22]:
print ('release do projeto=', dados[22][97])

release do projeto= 1185.0


In [23]:
print ('due do projeto=', dados[23][97])

due do projeto= 1680.0


In [24]:
(77 + 200) - (1531 - 1438 + 1) +60

243

In [25]:
%%time

c = 'lucro'

f, gastos, s, tempo, t_medio, feas = RodarHeuristica(arquivo=arq, criterio=c, print_sol=True)

projeto  57  sendo inserido na sonda 0  de  1408.0  até  1680 , sendo setup= 78.0 , e proc= 195.0  delta_setup= 0
projeto  30  sendo inserido na sonda 1  de  1394.0  até  1680 , sendo setup= 92.0 , e proc= 195.0  delta_setup= 0
projeto  77  sendo inserido na sonda 2  de  1385.0  até  1680 , sendo setup= 119.0 , e proc= 177.0  delta_setup= 0
projeto  70  sendo inserido na sonda 3  de  1287.0  até  1503.0 , sendo setup= 84.0 , e proc= 133.0  delta_setup= 0
projeto  42  sendo inserido na sonda 4  de  744.0  até  1014.0 , sendo setup= 85.0 , e proc= 186.0  delta_setup= 0
projeto  43  sendo inserido na sonda 5  de  870.0  até  1234.0 , sendo setup= 84.0 , e proc= 281.0  delta_setup= 0
projeto  27  sendo inserido na sonda 6  de  113.0  até  424.0 , sendo setup= 93.0 , e proc= 219.0  delta_setup= 0
projeto  87  sendo inserido na sonda 7  de  1039.0  até  1267.0 , sendo setup= 120.0 , e proc= 109.0  delta_setup= 0
projeto  46  sendo inserido na sonda 8  de  1441.0  até  1664.0 , sendo setup= 1

In [26]:
%%time

c = 'lucro/custo'

f, gastos, s, tempo, t_medio, feas = RodarHeuristica(arquivo=arq, criterio=c, print_sol=True)

projeto  57  sendo inserido na sonda 0  de  1408.0  até  1680 , sendo setup= 78.0 , e proc= 195.0  delta_setup= 0
projeto  70  sendo inserido na sonda 1  de  1281.0  até  1503.0 , sendo setup= 90.0 , e proc= 133.0  delta_setup= 0
projeto  30  sendo inserido na sonda 2  de  1375.0  até  1680 , sendo setup= 111.0 , e proc= 195.0  delta_setup= 0
projeto  77  sendo inserido na sonda 3  de  1422.0  até  1680 , sendo setup= 82.0 , e proc= 177.0  delta_setup= 0
projeto  42  sendo inserido na sonda 4  de  744.0  até  1014.0 , sendo setup= 85.0 , e proc= 186.0  delta_setup= 0
projeto  87  sendo inserido na sonda 5  de  1075.0  até  1267.0 , sendo setup= 84.0 , e proc= 109.0  delta_setup= 0
projeto  29  sendo inserido na sonda 6  de  50.0  até  172.0 , sendo setup= 50.0 , e proc= 73.0  delta_setup= 0
projeto  24  sendo inserido na sonda 7  de  1466.0  até  1680 , sendo setup= 119.0 , e proc= 96.0  delta_setup= 0
projeto  34  sendo inserido na sonda 8  de  1561.0  até  1680 , sendo setup= 56.0 , 

In [27]:
%%time

c = 'lucro/duracao'

f, gastos, s, tempo, t_medio, feas = RodarHeuristica(arquivo=arq, criterio=c, print_sol=True)

projeto  57  sendo inserido na sonda 0  de  1408.0  até  1680 , sendo setup= 78.0 , e proc= 195.0  delta_setup= 0
projeto  70  sendo inserido na sonda 1  de  1281.0  até  1503.0 , sendo setup= 90.0 , e proc= 133.0  delta_setup= 0
projeto  30  sendo inserido na sonda 2  de  1375.0  até  1680 , sendo setup= 111.0 , e proc= 195.0  delta_setup= 0
projeto  77  sendo inserido na sonda 3  de  1422.0  até  1680 , sendo setup= 82.0 , e proc= 177.0  delta_setup= 0
projeto  29  sendo inserido na sonda 4  de  50.0  até  172.0 , sendo setup= 50.0 , e proc= 73.0  delta_setup= 0
projeto  34  sendo inserido na sonda 5  de  1563.0  até  1680 , sendo setup= 54.0 , e proc= 64.0  delta_setup= 0
projeto  87  sendo inserido na sonda 6  de  1075.0  até  1267.0 , sendo setup= 84.0 , e proc= 109.0  delta_setup= 0
projeto  24  sendo inserido na sonda 7  de  1466.0  até  1680 , sendo setup= 119.0 , e proc= 96.0  delta_setup= 0
projeto  42  sendo inserido na sonda 8  de  744.0  até  1014.0 , sendo setup= 85.0 , e

In [28]:
%%time

c = 'custo'

f, gastos, s, tempo, t_medio, feas = RodarHeuristica(arquivo=arq, criterio=c, print_sol=True)

projeto  10  sendo inserido na sonda 0  de  659.0  até  808.0 , sendo setup= 99.0 , e proc= 51.0  delta_setup= 0
projeto  34  sendo inserido na sonda 1  de  1588.0  até  1680 , sendo setup= 29.0 , e proc= 64.0  delta_setup= 0
projeto  93  sendo inserido na sonda 2  de  47.0  até  171.0 , sendo setup= 58.0 , e proc= 67.0  delta_setup= 0
projeto  24  sendo inserido na sonda 3  de  1503.0  até  1680 , sendo setup= 82.0 , e proc= 96.0  delta_setup= 0
projeto  29  sendo inserido na sonda 4  de  50.0  até  172.0 , sendo setup= 50.0 , e proc= 73.0  delta_setup= 0
projeto  32  sendo inserido na sonda 5  de  1055.0  até  1185.0 , sendo setup= 58.0 , e proc= 73.0  delta_setup= 0
projeto  18  sendo inserido na sonda 6  de  1485.0  até  1680 , sendo setup= 115.0 , e proc= 81.0  delta_setup= 0
projeto  87  sendo inserido na sonda 7  de  1039.0  até  1267.0 , sendo setup= 120.0 , e proc= 109.0  delta_setup= 0
projeto  80  sendo inserido na sonda 8  de  1225.0  até  1437.0 , sendo setup= 101.0 , e pr

In [29]:
%%time

c = 'duracao'

f, gastos, s, tempo, t_medio, feas = RodarHeuristica(arquivo=arq, criterio=c, print_sol=True)

projeto  10  sendo inserido na sonda 0  de  659.0  até  808.0 , sendo setup= 99.0 , e proc= 51.0  delta_setup= 0
projeto  34  sendo inserido na sonda 1  de  1588.0  até  1680 , sendo setup= 29.0 , e proc= 64.0  delta_setup= 0
projeto  93  sendo inserido na sonda 2  de  47.0  até  171.0 , sendo setup= 58.0 , e proc= 67.0  delta_setup= 0
projeto  29  sendo inserido na sonda 3  de  50.0  até  172.0 , sendo setup= 50.0 , e proc= 73.0  delta_setup= 0
projeto  32  sendo inserido na sonda 4  de  1060.0  até  1185.0 , sendo setup= 53.0 , e proc= 73.0  delta_setup= 0
projeto  18  sendo inserido na sonda 5  de  1485.0  até  1680 , sendo setup= 115.0 , e proc= 81.0  delta_setup= 0
projeto  24  sendo inserido na sonda 6  de  1503.0  até  1680 , sendo setup= 82.0 , e proc= 96.0  delta_setup= 0
projeto  72  sendo inserido na sonda 7  de  1531.0  até  1680 , sendo setup= 52.0 , e proc= 98.0  delta_setup= 0
projeto  87  sendo inserido na sonda 8  de  1039.0  até  1267.0 , sendo setup= 120.0 , e proc= 

In [30]:
%%time

d = "./instancias"

resultados, df_resultados = RodarHeuristicaSensibilidade(diretorio=d, criterios=None)

processando arquivo:  ./instancias/instancia_500projetos_10sondas_delta_t1.dat
projeto  416  sendo inserido na sonda 0  de  1597.0  até  1680 , sendo setup= 27.0 , e proc= 57.0  delta_setup= 0
projeto  35  sendo inserido na sonda 1  de  1189.0  até  1405.0 , sendo setup= 56.0 , e proc= 161.0  delta_setup= 0
projeto  207  sendo inserido na sonda 2  de  1600.0  até  1680 , sendo setup= 30.0 , e proc= 51.0  delta_setup= 0
projeto  199  sendo inserido na sonda 3  de  569.0  até  770.0 , sendo setup= 51.0 , e proc= 151.0  delta_setup= 0
projeto  323  sendo inserido na sonda 4  de  1548.0  até  1680 , sendo setup= 83.0 , e proc= 50.0  delta_setup= 0
projeto  186  sendo inserido na sonda 5  de  1566.0  até  1680 , sendo setup= 54.0 , e proc= 61.0  delta_setup= 0
projeto  468  sendo inserido na sonda 6  de  1592.0  até  1680 , sendo setup= 26.0 , e proc= 63.0  delta_setup= 0
projeto  154  sendo inserido na sonda 7  de  443.0  até  617.0 , sendo setup= 31.0 , e proc= 144.0  delta_setup= 0
proje

projeto  416  sendo inserido na sonda 0  de  1597.0  até  1680 , sendo setup= 27.0 , e proc= 57.0  delta_setup= 0
projeto  207  sendo inserido na sonda 1  de  1611.0  até  1680 , sendo setup= 19.0 , e proc= 51.0  delta_setup= 0
projeto  323  sendo inserido na sonda 2  de  1548.0  até  1680 , sendo setup= 83.0 , e proc= 50.0  delta_setup= 0
projeto  177  sendo inserido na sonda 3  de  1550.0  até  1680 , sendo setup= 80.0 , e proc= 51.0  delta_setup= 0
projeto  186  sendo inserido na sonda 4  de  1591.0  até  1680 , sendo setup= 29.0 , e proc= 61.0  delta_setup= 0
projeto  397  sendo inserido na sonda 5  de  1545.0  até  1680 , sendo setup= 79.0 , e proc= 57.0  delta_setup= 0
projeto  475  sendo inserido na sonda 6  de  1582.0  até  1680 , sendo setup= 53.0 , e proc= 46.0  delta_setup= 0
projeto  468  sendo inserido na sonda 7  de  1592.0  até  1680 , sendo setup= 26.0 , e proc= 63.0  delta_setup= 0
projeto  151  sendo inserido na sonda 8  de  1582.0  até  1680 , sendo setup= 59.0 , e p

projeto  435  sendo inserido na sonda 0  de  292.0  até  431.0 , sendo setup= 47.0 , e proc= 93.0  delta_setup= 0.0
projeto  282  sendo inserido na sonda 5  de  1300.0  até  1399.0 , sendo setup= 54.0 , e proc= 46.0  delta_setup= 37.0
projeto  261  sendo inserido na sonda 4  de  66.0  até  316.0 , sendo setup= 62.0 , e proc= 189.0  delta_setup= 33.0
projeto  294  sendo inserido na sonda 2  de  1273.0  até  1307.0 , sendo setup= 7.0 , e proc= 28.0  delta_setup= 3.0
projeto  153  sendo inserido na sonda 2  de  358.0  até  464.0 , sendo setup= 50.0 , e proc= 57.0  delta_setup= -15.0
projeto  21  sendo inserido na sonda 9  de  853.0  até  980.0 , sendo setup= 24.0 , e proc= 104.0  delta_setup= -10.0
realocando: projeto  80  sendo inserido na sonda 8  na janela de  822.0  até  937.0 com delta_min= 85.0  delta_prev= 0  delta_next= 92.0  e delta_setup 29.0 , sendo setup= 46.0 , e proc= 99.0
projeto  376  sendo inserido na sonda 5  de  1254.0  até  1345.0 , sendo setup= 47.0 , e proc= 45.0  de

projeto  380  sendo inserido na sonda 0  de  137.0  até  349.0 , sendo setup= 55.0 , e proc= 158.0  delta_setup= 12.0
projeto  291  sendo inserido na sonda 3  de  172.0  até  397.0 , sendo setup= 90.0 , e proc= 136.0  delta_setup= 25.0
projeto  138  sendo inserido na sonda 4  de  202.0  até  424.0 , sendo setup= 46.0 , e proc= 177.0  delta_setup= 38.0
projeto  89  sendo inserido na sonda 7  de  720.0  até  869.0 , sendo setup= 41.0 , e proc= 109.0  delta_setup= 22.0
função objetivo correta
orçamento correto
unicidade correta
janelas de tempo corretas
setup correto
projeto  213  sendo inserido na sonda 0  de  344.0  até  459.0 , sendo setup= 88.0 , e proc= 28.0  delta_setup= 0
projeto  294  sendo inserido na sonda 1  de  1268.0  até  1377.0 , sendo setup= 82.0 , e proc= 28.0  delta_setup= 0
projeto  392  sendo inserido na sonda 2  de  1634.0  até  1680 , sendo setup= 19.0 , e proc= 28.0  delta_setup= 0
projeto  43  sendo inserido na sonda 3  de  1552.0  até  1680 , sendo setup= 83.0 , e

projeto  213  sendo inserido na sonda 0  de  344.0  até  459.0 , sendo setup= 88.0 , e proc= 28.0  delta_setup= 0
projeto  294  sendo inserido na sonda 1  de  1268.0  até  1377.0 , sendo setup= 82.0 , e proc= 28.0  delta_setup= 0
projeto  392  sendo inserido na sonda 2  de  1634.0  até  1680 , sendo setup= 19.0 , e proc= 28.0  delta_setup= 0
projeto  460  sendo inserido na sonda 3  de  1531.0  até  1680 , sendo setup= 115.0 , e proc= 35.0  delta_setup= 0
projeto  377  sendo inserido na sonda 4  de  1628.0  até  1680 , sendo setup= 14.0 , e proc= 39.0  delta_setup= 0
projeto  151  sendo inserido na sonda 5  de  1625.0  até  1680 , sendo setup= 16.0 , e proc= 40.0  delta_setup= 0
projeto  391  sendo inserido na sonda 6  de  384.0  até  512.0 , sendo setup= 88.0 , e proc= 41.0  delta_setup= 0
projeto  118  sendo inserido na sonda 7  de  1556.0  até  1680 , sendo setup= 83.0 , e proc= 42.0  delta_setup= 0
projeto  329  sendo inserido na sonda 8  de  9.0  até  64.0 , sendo setup= 14.0 , e p

...

projeto  467  sendo inserido na sonda 8  de  110.0  até  159.0 , sendo setup= 12.0 , e proc= 38.0  delta_setup= 12.0
projeto  388  sendo inserido na sonda 9  de  38.0  até  49.0 , sendo setup= 6.0 , e proc= 6.0  delta_setup= 0.0
realocando: projeto  793  sendo inserido na sonda 0  na janela de  173.0  até  176.0 com delta_min= 2.0  delta_prev= 0  delta_next= 2.0  e delta_setup 6.0 , sendo setup= 6.0 , e proc= 4.0
projeto  827  sendo inserido na sonda 1  de  60.0  até  83.0 , sendo setup= 11.0 , e proc= 13.0  delta_setup= -7.0
projeto  514  sendo inserido na sonda 7  de  95.0  até  103.0 , sendo setup= 2.0 , e proc= 7.0  delta_setup= -2.0
projeto  261  sendo inserido na sonda 2  de  146.0  até  167.0 , sendo setup= 11.0 , e proc= 11.0  delta_setup= 2.0
projeto  272  sendo inserido na sonda 6  de  231.0  até  239.0 , sendo setup= 4.0 , e proc= 5.0  delta_setup= 0
projeto  62  sendo inserido na sonda 3  de  130.0  até  139.0 , sendo setup= 3.0 , e proc= 7.0  delta_setup= 1.0
projeto  223 

projeto  823  sendo inserido na sonda 1  de  99.0  até  123.0 , sendo setup= 10.0 , e proc= 15.0  delta_setup= 2.0
projeto  792  sendo inserido na sonda 4  de  138.0  até  144.0 , sendo setup= 3.0 , e proc= 4.0  delta_setup= -2.0
projeto  817  sendo inserido na sonda 8  de  194.0  até  202.0 , sendo setup= 4.0 , e proc= 5.0  delta_setup= 4.0
projeto  556  sendo inserido na sonda 9  de  99.0  até  119.0 , sendo setup= 10.0 , e proc= 11.0  delta_setup= 10.0
projeto  549  sendo inserido na sonda 6  de  86.0  até  98.0 , sendo setup= 5.0 , e proc= 8.0  delta_setup= 3.0
projeto  338  sendo inserido na sonda 4  de  15.0  até  37.0 , sendo setup= 13.0 , e proc= 10.0  delta_setup= -2.0
projeto  562  sendo inserido na sonda 0  de  88.0  até  92.0 , sendo setup= 0.0 , e proc= 5.0  delta_setup= 0.0
projeto  946  sendo inserido na sonda 8  de  119.0  até  144.0 , sendo setup= 11.0 , e proc= 15.0  delta_setup= 9.0
projeto  989  sendo inserido na sonda 0  de  34.0  até  52.0 , sendo setup= 11.0 , e 

projeto  4  sendo inserido na sonda 9  de  129.0  até  139.0 , sendo setup= 5.0 , e proc= 6.0  delta_setup= -5.0
projeto  659  sendo inserido na sonda 4  de  82.0  até  103.0 , sendo setup= 13.0 , e proc= 9.0  delta_setup= 10.0
projeto  241  sendo inserido na sonda 2  de  2.0  até  13.0 , sendo setup= 6.0 , e proc= 6.0  delta_setup= 6.0
projeto  410  sendo inserido na sonda 5  de  147.0  até  159.0 , sendo setup= 7.0 , e proc= 6.0  delta_setup= 0.0
projeto  216  sendo inserido na sonda 7  de  173.0  até  182.0 , sendo setup= 4.0 , e proc= 6.0  delta_setup= 1.0
projeto  992  sendo inserido na sonda 3  de  179.0  até  188.0 , sendo setup= 4.0 , e proc= 6.0  delta_setup= 3.0
projeto  323  sendo inserido na sonda 6  de  89.0  até  110.0 , sendo setup= 13.0 , e proc= 9.0  delta_setup= -11.0
projeto  563  sendo inserido na sonda 9  de  121.0  até  132.0 , sendo setup= 6.0 , e proc= 6.0  delta_setup= 4.0
projeto  699  sendo inserido na sonda 0  de  183.0  até  198.0 , sendo setup= 6.0 , e pro

projeto  810  sendo inserido na sonda 0  de  8.0  até  32.0 , sendo setup= 12.0 , e proc= 13.0  delta_setup= 11.0
projeto  788  sendo inserido na sonda 7  de  3.0  até  30.0 , sendo setup= 10.0 , e proc= 18.0  delta_setup= 9.0
função objetivo correta
orçamento correto
unicidade correta
janelas de tempo corretas
setup correto
projeto  61  sendo inserido na sonda 0  de  221.0  até  234.0 , sendo setup= 10.0 , e proc= 4.0  delta_setup= 0
projeto  112  sendo inserido na sonda 1  de  152.0  até  159.0 , sendo setup= 4.0 , e proc= 4.0  delta_setup= 0
projeto  113  sendo inserido na sonda 2  de  230.0  até  240 , sendo setup= 7.0 , e proc= 4.0  delta_setup= 0
projeto  156  sendo inserido na sonda 3  de  231.0  até  240 , sendo setup= 6.0 , e proc= 4.0  delta_setup= 0
projeto  208  sendo inserido na sonda 4  de  21.0  até  36.0 , sendo setup= 12.0 , e proc= 4.0  delta_setup= 0
projeto  218  sendo inserido na sonda 5  de  40.0  até  50.0 , sendo setup= 7.0 , e proc= 4.0  delta_setup= 0
projeto 

projeto  959  sendo inserido na sonda 2  de  171.0  até  187.0 , sendo setup= 10.0 , e proc= 7.0  delta_setup= 10.0
projeto  12  sendo inserido na sonda 0  de  102.0  até  116.0 , sendo setup= 7.0 , e proc= 8.0  delta_setup= 7.0
projeto  50  sendo inserido na sonda 4  de  113.0  até  121.0 , sendo setup= 1.0 , e proc= 8.0  delta_setup= -1.0
projeto  73  sendo inserido na sonda 5  de  172.0  até  184.0 , sendo setup= 5.0 , e proc= 8.0  delta_setup= 5.0
projeto  78  sendo inserido na sonda 8  de  130.0  até  141.0 , sendo setup= 4.0 , e proc= 8.0  delta_setup= 0.0
projeto  151  sendo inserido na sonda 6  de  32.0  até  45.0 , sendo setup= 6.0 , e proc= 8.0  delta_setup= -6.0
projeto  138  sendo inserido na sonda 7  de  146.0  até  155.0 , sendo setup= 2.0 , e proc= 8.0  delta_setup= 2.0
projeto  139  sendo inserido na sonda 4  de  122.0  até  132.0 , sendo setup= 3.0 , e proc= 8.0  delta_setup= -2.0
projeto  401  sendo inserido na sonda 1  de  94.0  até  107.0 , sendo setup= 6.0 , e proc

In [31]:
df_resultados

,Instancia,Viabilidade,Criterio guloso,Tempo total (s),Tempo medio (s),Valor da função objetivo
0,/instancia_500projetos_10sondas_delta_t1.dat,feasible,lucro,0.313347,0.156675,70044.800124
1,/instancia_500projetos_10sondas_delta_t1.dat,feasible,lucro/custo,0.277187,0.138595,69689.051175
2,/instancia_500projetos_10sondas_delta_t1.dat,feasible,lucro/duracao,0.222060,0.111031,68922.472035
3,/instancia_500projetos_10sondas_delta_t1.dat,feasible,custo,0.198141,0.099071,51990.393076
4,/instancia_500projetos_10sondas_delta_t1.dat,feasible,duracao,0.215643,0.107822,53131.712580
...,...,...,...,...,...,...
325,/instancia_10projetos_2sondas_delta_t7.dat,feasible,lucro,0.011576,0.005789,618.164588
326,/instancia_10projetos_2sondas_delta_t7.dat,feasible,lucro/custo,0.009181,0.004591,618.164588
327,/instancia_10projetos_2sondas_delta_t7.dat,feasible,lucro/duracao,0.010339,0.005171,618.164588
328,/instancia_10projetos_2sondas_delta_t7.dat,feasible,custo,0.010286,0.005144,618.164588


In [32]:
df_resultados['Viabilidade'].value_counts()

feasible    330
Name: Viabilidade, dtype: int64

In [33]:
df_resultados[df_resultados['Viabilidade']=='not feasible']

,Instancia,Viabilidade,Criterio guloso,Tempo total (s),Tempo medio (s),Valor da função objetivo


In [34]:
df_resultados.to_csv(path_or_buf='resultados.csv', header=True, index=False, decimal='.')